In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import package

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torchsummary import summary
from torch import optim

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os

# Cross Validation
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit

# Scheduler
from torch.optim.lr_scheduler import StepLR

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
from torchsummary import summary
import time
import copy

# submission
from tqdm import tqdm
import pandas as pd
from pandas import DataFrame

## **Define ResNeXt model**

In [3]:
class BottleNeck(nn.Module):
    expansion = 4
    Cardinality = 32 # group 수
    Basewidth = 64 # bottleneck 채널이 64이면 group convolution의 채널은 depth가 됩니다.
    Depth = 4 # basewidth일 때, group convolution의 채널 수
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        C = BottleNeck.Cardinality
        D = int(BottleNeck.Depth * out_channels / BottleNeck.Basewidth)

        self.conv_residual = nn.Sequential(
            nn.Conv2d(in_channels, C * D, 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(C*D),
            nn.ReLU(),
            nn.Conv2d(C*D, C*D, 3, stride=stride, padding=1, groups=BottleNeck.Cardinality, bias=False),
            nn.BatchNorm2d(C*D),
            nn.ReLU(),
            nn.Conv2d(C*D, out_channels * BottleNeck.expansion, 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion)
        )

        # y = x + F(x)일 때 x와 F(x)의 채널을 맞춤
        self.conv_shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.conv_shortcut = nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, 1, stride=stride, padding=0)

    def forward(self, x):
        x = self.conv_residual(x) + self.conv_shortcut(x)
        return x


# ResNext
class ResNeXt(nn.Module):
    def __init__(self, nblocks, num_classes=10, init_weights=True):
        super().__init__()
        self.init_weights=init_weights
        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, stride=2, padding=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2, padding=1)
        )

        self.conv2 = self._make_res_block(nblocks[0], 64, 1)
        self.conv3 = self._make_res_block(nblocks[1], 128, 2)
        self.conv4 = self._make_res_block(nblocks[2], 256, 2)
        self.conv5 = self._make_res_block(nblocks[3], 512, 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.linear = nn.Linear(512 * BottleNeck.expansion, num_classes)

        # weights initialization
        if self.init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

    def _make_res_block(self, nblock, out_channels, stride):
        strides = [stride] + [1] * (nblock-1)
        res_block = nn.Sequential()
        for i, stride in enumerate(strides):
            res_block.add_module('dens_layer_{}'.format(i), BottleNeck(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * BottleNeck.expansion
        return res_block

    # weights initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def ResNeXt50():
    return ResNeXt([3, 4, 6, 3], num_classes=num_classes)

## **Utils**

In [4]:
class AverageMeter(object):
    r"""Computes and stores the average and current value
    """
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    r"""Computes the accuracy over the $k$ top predictions for the specified values of k
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # _, pred = output.topk(maxk, 1, True, True)
        # pred = pred.t()
        # correct = pred.eq(target.view(1, -1).expand_as(pred))

        # faster topk (ref: https://github.com/pytorch/pytorch/issues/22812)
        _, idx = output.sort(descending=True)
        pred = idx[:,:maxk]
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

## **Cutout: Main Code for Applying Cutout data augmentation**

In [5]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.

    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

## **Parameter Setting**

In [6]:
dataset = 'cifar10' # cifar10 or cifar100
model = 'ResNeXt50'
batch_size = 64  # Input batch size for training (default: 128)
epochs = 6 # Number of epochs to train (default: 200)
learning_rate = 0.01 # Learning rate
data_augmentation = False # Traditional data augmentation such as augmantation by flipping and cropping?
fold_num = 5 # k-fold validation
path2submission = f'/content/drive/Shareddrives/Data/Kaggle/GEK6189_CIFAR-10_Competition_2022-1/submission/{model}.csv'    # route for submission .csv file
path2weights = f'/content/drive/Shareddrives/Data/Kaggle/GEK6189_CIFAR-10_Competition_2022-1/models/{model}'

cutout = False # Apply Cutout?
if cutout:
    n_holes = 1 # Number of holes to cut out from image
    length = 16 # Length of the holes

seed = 0 # Random seed (default: 0)
print_freq = 100
cuda = torch.cuda.is_available()
cudnn.benchmark = True  # Should make training should go faster for large models

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

test_id = dataset + '_' + model

In [7]:
# Image Preprocessing
normalize = transforms.Normalize(mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

# train
train_transform = transforms.Compose([])

train_transform.transforms.append(transforms.Resize((64, 64)))
if data_augmentation:
    #train_transform.transforms.append(transforms.RandomCrop(299, 299))
    train_transform.transforms.append(transforms.RandomHorizontalFlip())
train_transform.transforms.append(transforms.ToTensor())
train_transform.transforms.append(normalize)

if cutout:
    train_transform.transforms.append(Cutout(n_holes=n_holes, length=length))

# test
test_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    normalize])


# load dataset
if dataset == 'cifar10':
    num_classes = 10
    train_dataset = datasets.CIFAR10(root='data/',
                                     train=True,
                                     transform=train_transform,
                                     download=True)

    test_dataset = datasets.CIFAR10(root='data/',
                                    train=False,
                                    transform=test_transform,
                                    download=True)

Files already downloaded and verified
Files already downloaded and verified


## **Main Training**

In [8]:
def train(train_loader, epoch, model, optimizer, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, losses,
                             top1, top5, prefix="Epoch: [{}]".format(epoch))
    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        input = input.cuda()
        target = target.cuda()
        #input = input.to(torch.device("cpu"))
        #target = target.to(torch.device("cpu"))

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss, accuracy 
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(acc1[0].item(), input.size(0))
        top5.update(acc5[0].item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            progress.print(i)

    print('==> Train Accuracy: Acc@1 {top1.avg:.3f} || Acc@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))
    return top1.avg

def valid(val_loader, model):
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    model.eval()
    for i,(input,target) in enumerate(val_loader):
        input = input.cuda()
        target = target.cuda()
        #input = input.to(torch.device("cpu"))
        #target = target.to(torch.device("cpu"))

        output = model(input)
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        top1.update(acc1[0].item(), input.size(0))
        top5.update(acc5[0].item(), input.size(0))
    print('==> Valid Accuracy:  Acc@1 {top1.avg:.3f} || Acc@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))
    return top1.avg

def test(test_loader, model):  
    output = None
    model.eval()
    for i, (input, target) in enumerate(tqdm(test_loader), 0):
        input = input.cuda()
        #target은 쓰지 않음.
        #target = target.cuda()
        if output is not None:
            output = torch.cat((output, model(input)), dim=0)
        else:
            output = model(input)
    return output

In [9]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=fold_num, shuffle=True)

In [10]:
# Model evaluation using K-fold cross-validation
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_dataset)):

    # Print
    print(f'\nFOLD {fold+1}')
    print('--------------------------------')

    # Ramdom sample elements from a given list of ids, not replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    train_loader = DataLoader(dataset = train_dataset,
                         batch_size=batch_size,
                         sampler = train_subsampler,
                         pin_memory=True,
                         num_workers=2)
    
    valid_loader = DataLoader(dataset = train_dataset,
                         batch_size=batch_size,
                         sampler = valid_subsampler,
                         pin_memory=True,
                         num_workers=2)
    
    # Setting
    if model == 'ResNeXt50':
        model = ResNeXt50().cuda()

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
    #criterion = torch.nn.CrossEntropyLoss(reduction='sum').cuda()
    criterion = torch.nn.CrossEntropyLoss(reduction='sum').cuda()

    # Training
    best_acc = 0
    for epoch in range(epochs):
        # Train for one epoch
        train(train_loader, epoch, model, optimizer, criterion)

        # Validation
        with torch.no_grad():
            val_acc = valid(valid_loader, model)

        # learning rate scheduling
        scheduler.step()
    
        # Save model for best accuracy
        if best_acc < val_acc:
            path2weights_fold = f'{path2weights}_fold{fold+1}.pth'  # route for model saving
            best_acc = val_acc
            torch.save(model.state_dict(), path2weights_fold)

    print(f"Best Top-1 Accuracy for fold{fold+1}: {best_acc}")


FOLD 1
--------------------------------
Epoch: [0][  0/625]	Time  1.986 ( 1.986)	Loss 1.6586e+02 (1.6586e+02)	Acc@1  10.94 ( 10.94)	Acc@5  59.38 ( 59.38)
Epoch: [0][100/625]	Time  0.086 ( 0.106)	Loss 2.3565e+02 (6.5771e+03)	Acc@1  15.62 ( 16.32)	Acc@5  67.19 ( 60.26)
Epoch: [0][200/625]	Time  0.087 ( 0.098)	Loss 1.2724e+02 (3.3871e+03)	Acc@1  29.69 ( 21.28)	Acc@5  81.25 ( 68.91)
Epoch: [0][300/625]	Time  0.089 ( 0.095)	Loss 1.3966e+02 (2.3064e+03)	Acc@1  39.06 ( 24.71)	Acc@5  85.94 ( 73.42)
Epoch: [0][400/625]	Time  0.090 ( 0.094)	Loss 1.1409e+02 (1.7612e+03)	Acc@1  35.94 ( 27.51)	Acc@5  84.38 ( 76.50)
Epoch: [0][500/625]	Time  0.090 ( 0.093)	Loss 1.2408e+02 (1.4330e+03)	Acc@1  35.94 ( 29.45)	Acc@5  89.06 ( 78.57)
Epoch: [0][600/625]	Time  0.091 ( 0.093)	Loss 1.2239e+02 (1.2128e+03)	Acc@1  28.12 ( 31.32)	Acc@5  87.50 ( 80.35)
==> Train Accuracy: Acc@1 31.683 || Acc@5 80.705
==> Valid Accuracy:  Acc@1 42.820 || Acc@5 91.620
Epoch: [1][  0/625]	Time  0.508 ( 0.508)	Loss 1.2247e+02 (1.22

## **Test with ensemble**

In [11]:
test_loader = DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          pin_memory=True,
                                          num_workers=2)

In [12]:
# ensemble
with torch.no_grad():
    output = None
    predictions_list = []
   
    for fold in range(fold_num):
        print(f'[Fold {fold+1}]')

        if output is not None:
            del output   # del output : memory free
        else :
            pass
        path2weights_fold = f'{path2weights}_fold{fold+1}.pth'

        if model == "ResNeXt50":
            model = ResNeXt50().cuda()
            
        model.load_state_dict(torch.load(path2weights_fold))

        output = test(test_loader, model)
        output = output.view(output.shape[0], output.shape[1], 1).cpu() # 각 fold별 output의 평균을 내기 위해 3번째 dimension 추가
        predictions_list.append(output)

    predictions_array = np.concatenate(predictions_list, axis=2)
    predictions_mean = predictions_array.mean(axis = 2)
    predictions_mean = torch.from_numpy(predictions_mean)
    predictData = torch.argmax(predictions_mean, 1)

    classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    lst = []
    for Id, predict in enumerate(predictData, 1):
        # tensor형태의 predict를 정수의 prediction으로 바꿈
        x = predict.to("cpu").numpy()
        x = x.astype(int)
        Category = classes[x]
        lst.append([Id, Category])

    df = DataFrame(lst, columns=['Id', 'Category'])
    df.to_csv(path2submission, index=False, encoding='cp949')

[Fold 1]


100%|██████████| 157/157 [00:04<00:00, 32.30it/s]


[Fold 2]


100%|██████████| 157/157 [00:04<00:00, 32.82it/s]


[Fold 3]


100%|██████████| 157/157 [00:04<00:00, 32.55it/s]


[Fold 4]


100%|██████████| 157/157 [00:04<00:00, 32.26it/s]


[Fold 5]


100%|██████████| 157/157 [00:04<00:00, 32.09it/s]
